In [ ]:
!uv pip install plotly nbformat pandas

In [3]:
df = pd.read_excel("copernicus_climate_20260202.xlsx")#, encoding="latin-1")
df

,Title,Subtitle,CELEX number,Initiating document,Procedure reference,Initial reference,Adopted acts,Current consolidated version,Latest consolidated version,Procedure status,...,Author,Date of effect,Date of end of validity,Date of transposition,Applicant/Appellant,Defendant/Other parties to the proceedings,In force indicator,Number of pages,Publication Reference,Unnamed: 35
0,Proposal for a DECISION OF THE COUNCIL on esta...,ST 7911 2019 REV 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Council of the European Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Proposal for a DECISION OF THE COUNCIL on esta...,ST 7911 2019 INIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Council of the European Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Opinion - European Economic and Social Committ...,EESC-2021-06049-AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,European Economic and Social Committee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Opinion - European Economic and Social Committ...,EESC-2023-01246-AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,European Economic and Social Committee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Opinion - European Economic and Social Committ...,EESC-2022-01215-AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,European Economic and Social Committee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,Proposal for a REGULATION OF THE EUROPEAN PARL...,ST 8794 2020 INIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Council of the European Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1268,Written questions by Members of the European P...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,European Parliament,NaN,NaN,NaN,NaN,NaN,NaN,297,"OJ C 296, , p. 1–297\n\t\t (BG, ES, CS, DA, DE...",NaN
1269,Written questions by Members of the European P...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,European Parliament,NaN,NaN,NaN,NaN,NaN,NaN,301,"OJ C 273, , p. 1–301\n\t\t (BG, ES, CS, DA, DE...",NaN
1270,Written questions by Members of the European P...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,European Parliament,NaN,NaN,NaN,NaN,NaN,NaN,311,"OJ C 237, , p. 1–311\n\t\t (BG, ES, CS, DA, DE...",NaN


In [1]:
import pandas as pd
import plotly.graph_objects as go

import pandas as pd 
df = pd.read_excel("copernicus_climate_20260202.xlsx")#, encoding="latin-1")
df = df.drop_duplicates(subset=['Title', 'Subtitle', 'CELEX number'], keep='first')

important_eu_document_types = [
    "Treaty",
    "Regulation",
    "Directive",
    "Decision",
    "Delegated regulation",
    "Implementing regulation",
    "Implementing decision",
    "Communication",
    "Joint communication",
    "Strategy",
]

df = df.loc[df["Form"].isin(important_eu_document_types)]

In [2]:
import pandas as pd
import plotly.graph_objects as go

# 1. Prepare the data
df['Date of document'] = pd.to_datetime(df['Date of document'])
df['Form'] = df['Form'].fillna('Unknown').astype(str)

unique_forms = sorted(df['Form'].unique().tolist())
num_forms = len(unique_forms)

# Function to aggregate data
def get_pivoted_data(freq):
    if freq == 'D':
        grouper = df['Date of document'].dt.floor('D')
    elif freq == 'M':
        grouper = df['Date of document'].dt.to_period('M').dt.to_timestamp()
    else: # Yearly
        grouper = df['Date of document'].dt.to_period('Y').dt.to_timestamp()
    
    pivoted = df.groupby([grouper, 'Form']).size().unstack(fill_value=0)
    for col in unique_forms:
        if col not in pivoted.columns: pivoted[col] = 0
    return pivoted[unique_forms]

daily_data = get_pivoted_data('D')
monthly_data = get_pivoted_data('M')
yearly_data = get_pivoted_data('Y')

# 2. Create the Figure
fig = go.Figure()

# Function to add bar traces AND a total sum trace
def add_traces(data, is_visible):
    # Add the individual bars for each Form
    for form in unique_forms:
        fig.add_trace(go.Bar(
            x=data.index,
            y=data[form],
            name=form,
            text=data[form].apply(lambda x: x if x > 0 else ""),
            textposition='auto',
            visible=is_visible,
            legendgroup=form,
            showlegend=True
        ))
    
    # CALCULATE AND ADD TOTALS:
    # We use a Scatter trace in 'text' mode to place the total sum at the top
    totals = data.sum(axis=1)
    fig.add_trace(go.Scatter(
        x=data.index,
        y=totals,
        mode='text',
        text=totals,
        textposition='top center',
        textfont=dict(size=12, color="black", family="Arial Black"),
        name='Total',
        visible=is_visible,
        showlegend=False, # We don't want "Total" in the legend group
        hoverinfo='skip'  # Don't let the total trace interfere with bar tooltips
    ))

# Add traces in order. Each group now has (num_forms + 1) traces
add_traces(daily_data, False)    
add_traces(monthly_data, False)  
add_traces(yearly_data, True)    

# 3. Define Visibility Lists (Each set is now num_forms + 1)
per_group = num_forms + 1
vis_daily   = [True] * per_group + [False] * per_group + [False] * per_group
vis_monthly = [False] * per_group + [True] * per_group + [False] * per_group
vis_yearly  = [False] * per_group + [False] * per_group + [True] * per_group

# 4. Update Layout
fig.update_layout(
    height=850, 
    width=None,
    template="plotly_white",
    barmode='stack',
    title="Yearly Documents by Form",
    legend_title="Document Form",
    # Added extra padding at the top of the y-axis so totals don't get cut off
    yaxis=dict(range=[0, yearly_data.sum(axis=1).max() * 1.15]), 
    updatemenus=[
        {
            "active": 2,
            "buttons": [
                {
                    "label": "Daily",
                    "method": "update",
                    "args": [{"visible": vis_daily}, 
                             {"title": "Daily Documents by Form", "yaxis.range": [0, daily_data.sum(axis=1).max() * 1.2]}]
                },
                {
                    "label": "Monthly",
                    "method": "update",
                    "args": [{"visible": vis_monthly}, 
                             {"title": "Monthly Documents by Form", "yaxis.range": [0, monthly_data.sum(axis=1).max() * 1.2]}]
                },
                {
                    "label": "Yearly",
                    "method": "update",
                    "args": [{"visible": vis_yearly}, 
                             {"title": "Yearly Documents by Form", "yaxis.range": [0, yearly_data.sum(axis=1).max() * 1.2]}]
                },
            ],
            "direction": "down",
            "showactive": True,
            "x": 0,
            "y": 1.2
        }
    ]
)

# 5. Show and Export
fig.show()
fig.write_html("index.html", include_plotlyjs='cdn')